In [1]:
import json
import pandas as pd
from datasets import load_from_disk,DatasetDict, Dataset
import re
import random
from collections import OrderedDict
random.seed(42)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = './wiki_data/wikipedia_documents.json'
with open(file_path,"r") as f:
    wiki = json.load(f)

In [3]:
# 문장 자르기
def make_sentence(text):
    a=text.split('\n')
    tmp=[]
    for c in a:
        tmp += re.split(r'([.!?])',c)
    tmp2 = []
    special=['.','!','?']
    for i in range(len(tmp)-1):
        if tmp[i] in special:
            continue
        elif tmp[i] == '':
            continue
        else:
            if tmp[i+1] in special:
                tmp2.append(tmp[i]+tmp[i+1])
                i += 1
            else:
                tmp2.append(tmp[i])
    return tmp2

In [4]:
# ICT make for train_dataset
i=0
error_list=[]
titles = []
contexts = []
questions = []
answers = []
ids = []
for key, val in wiki.items():
    try:
        sentences=make_sentence(val['text'])
        target = random.randint(0,len(sentences)-1)
        target_senteces = ' '.join(sentences[target:target+2])
        target_context = ' '.join(sentences[:target]+sentences[target+2:])
        titles.append(val['title'])
        contexts.append(target_context)
        questions.append(target_senteces)
        ids.append(f'{i}')
        i += 1
    except:
        error_list.append(key)

wiki_df = pd.DataFrame({'title':titles,'context': contexts, 'question': questions, 'id': ids})
len(wiki_df)


60576

In [5]:
wiki_df = wiki_df.drop_duplicates(['context','question'],keep='first',ignore_index=True)
len(wiki_df)

60104

In [30]:
train_dataset = Dataset.from_pandas(wiki_df[:50001])
valid_dataset = Dataset.from_pandas(wiki_df[50000:])
datasets = DatasetDict({'train': train_dataset, 'validation': valid_dataset})
datasets.save_to_disk('wiki_ict_dataset')

In [7]:
wiki_ict_train_data=OrderedDict()
wiki_ict_valid_data=OrderedDict()
for idx,data in enumerate(wiki_df.iterrows()):
    data = dict(data[1])
    if idx <=50000:
        wiki_ict_train_data[idx] = data
    else:
        wiki_ict_valid_data[idx] = data
with open('./wiki_data/wiki_ict_train_data.json','w') as outfile:
    json.dump(wiki_ict_train_data,outfile,ensure_ascii=False)
with open('./wiki_data/wiki_ict_valid_data.json','w') as outfile:
    json.dump(wiki_ict_valid_data,outfile,ensure_ascii=False)

In [8]:
for error in error_list:
    print(wiki[error])

{'text': '가면팜사향고양이 · 가시두더지 · 가젤영양 · 가지뿔영양 · 갈기늑대 · 갈기쥐 · 갈라파고스물개 · 갈라파고스펭귄 · 갈색여우원숭이 · 강멧돼지 · 개 · 개미핥기 · 개코원숭이 · 거농원숭이 · 거미원숭이 · 검둥원숭이 · 검은고함원숭이 · 검은머리꼬리감기원숭이 · 검은수염고래 · 검은여우원숭이 · 게잡이물범 · 게잡이원숭이 · 고라니 · 고래 · 고릴라 · 고슴도치 · 고양이 · 고함원숭이 · 곰 · 관여우원숭이 · 관박쥐 · 귀신고래 ·그랜트가젤 · 그랜트얼룩말 · 그레비얼룩말 · 그레이하운드 · 그리슨족제비 · 금강산코박쥐 · 기린 · 긴귀주머니오소리 · 긴칼뿔오릭스 · 긴팔원숭이 · 꼬리감기원숭이 · 꿀먹이박쥐', 'corpus_source': '위키피디아', 'url': 'TODO', 'domain': None, 'title': '포유류의 목록', 'author': None, 'html': None, 'document_id': 674}
{'text': '대덕고등학교  대덕소프트웨어마이스터고등학교  대전가오고등학교  대전고등학교  대전공업고등학교  대전과학고등학교  대전관저고등학교  대전괴정고등학교  대전구봉고등학교  대전국제통상고등학교  대전노은고등학교  대전대성고등학교  대전대성여자고등학교  대전대신고등학교  대전도안고등학교  대전동산고등학교  대전동신과학고등학교  대전둔산여자고등학교  대전둔원고등학교  대전만년고등학교  대전반석고등학교  대전복수고등학교  대전생활과학고등학교  대전성모여자고등학교  대전송촌고등학교  대전시립중고등학교  대전신일여자고등학교  대전여자고등학교  대전여자상업고등학교  대전예술고등학교  대전외국어고등학교  대전용산고등학교  대전이문고등학교  대전전민고등학교  대전전자디자인고등학교  대전제일고등학교  대전중앙고등학교  대전지족고등학교  대전체육고등학교  대전한빛고등학교  동대전고등학교  동방고등학교  동아마이스터고등학교', 'corpus_source': '위키피디아', 'url': 'TODO', 'dom